In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['mart_djy_03_201412.txt',
 'mart_djy_03_201612.txt',
 'mart_djy_03_201712.txt',
 'mart_djy_03_201812.txt',
 'mart_djy_03_201912.txt',
 'mart_djy_03_202012.txt',
 'mart_djy_03_202102.txt',
 'MART_DJY_TITLE_201512.txt']

In [3]:
colnames_df = pd.read_excel(basedir + '데이터구조.xls', header=1)
print(colnames_df.shape)
colnames_df.head()

(77, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [4]:
colnames = colnames_df['컬럼 한글명'].tolist()
print(len(colnames))
colnames[:5]

77


['관리_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명']

In [5]:
colnames_df2 = pd.read_excel(basedir + '데이터구조2.xls', header=1)
print(colnames_df2.shape)
colnames_df2.head()

(77, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [6]:
colnames2 = colnames_df2['컬럼 한글명'].tolist()
print(len(colnames2))
colnames2[:5]

77


['관리_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명']

In [7]:
%%time
dfs_list = []
for i, file in enumerate(filenames):
    yearmonth = file.replace('.txt', '')[-6:]
    year = yearmonth[:4]
    print(yearmonth)
    
    if i == 0:
        df = pd.read_csv(basedir + file, sep='|', header=0, names=colnames)
        current_cols = df.columns.tolist()
        
        new_cols = []
        
        for i in range(len(current_cols)):
            if i <= 16:
                new_cols.append(current_cols[i])
            elif i == 17:
                new_cols.append('내진_설계_적용_여부')
            elif i <= 25:
                new_cols.append(current_cols[i-1])
            elif i == 26:
                new_cols.append('내진_능력')
            else:
                new_cols.append(current_cols[i-2])
        
        df.columns = new_cols
        
        colnames_75 = colnames.copy()
        colnames_75.remove('내진_설계_적용_여부')
        colnames_75.remove('내진_능력')
        
    elif i == 2:
        df = pd.read_csv(basedir + file, sep='|', header=0, encoding='euc-kr', names=colnames)
        df.columns = new_cols
    else:
        df = pd.read_csv(basedir + file, sep='|', encoding='euc-kr', header=None)
        if df.shape[1] == 77:
            df.columns = new_cols
            if year == '2020':
                df.columns = colnames
            elif year == '2021':
                df.columns = colnames2
        elif df.shape[1] == 75:
            df.columns = colnames_75
            
    df['기준년월'] = int(yearmonth)
        
    dfs_list.append(df)

201412


<string>:2: DtypeWarning: Columns (13,14,15,33,36,39,60,61,62,63) have mixed types.Specify dtype option on import or set low_memory=False.


201612


<string>:2: DtypeWarning: Columns (14,15,33,39,60,61,62,63,64,69) have mixed types.Specify dtype option on import or set low_memory=False.


201712


...

202012
202102


<string>:2: DtypeWarning: Columns (15,31,58,59,60,61,62,67) have mixed types.Specify dtype option on import or set low_memory=False.


201512


<string>:2: DtypeWarning: Columns (14,15,33,36,39,60,61,62,63,64,69) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 12min 19s


In [8]:
prac_df = dfs_list[6]
print(prac_df.shape)
prac_df.head()

(7849930, 78)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,내진_설계_적용_여부,내진_능력,기준년월
0,27710-2503,1,일반,2,일반건축물,대구광역시 달성군 화원읍 천내리 387-5번지,대구광역시 달성군 화원읍 비슬로512길 57-9,NaN,27710,25021.0,0.0,387,5,NaN,NaN,NaN,0,2.771042e+11,25001.0,0.0,57.0,9.0,NaN,1.0,부속건축물,0.0,2.22,0.00,2.22,2.22,0.00,99.0,기타구조,샌드위치판넬,04000,제2종근린생활시설,방앗간,90.0,기타지붕,판넬,0,1,0.0,1,0,0,0,1,2.22,2.22,0,0.0,0,0.0,0,0.0,0,0.0,19830325.0,NaN,19830325.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN,202102
1,28710-6007,1,일반,2,일반건축물,인천광역시 강화군 양도면 도장리 685-1번지,인천광역시 강화군 양도면 강화남로428번길 46-20,NaN,28710,35027.0,0.0,685,1,NaN,NaN,NaN,0,2.871043e+11,35001.0,0.0,46.0,20.0,NaN,0.0,주건축물,522.0,98.94,18.95,141.60,141.60,27.12,11.0,벽돌구조,조적조,01000,단독주택,주택,10.0,(철근)콘크리트,슬라브,0,1,6.6,2,0,0,0,0,0.00,141.60,0,0.0,0,0.0,0,0.0,0,0.0,19960604.0,19960605.0,19980529.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN,202102
2,28710-23420,1,일반,2,일반건축물,인천광역시 강화군 선원면 냉정리 916-1번지,인천광역시 강화군 선원면 중앙로 468,NaN,28710,31027.0,0.0,916,1,NaN,NaN,NaN,0,2.871032e+11,31002.0,0.0,468.0,0.0,NaN,0.0,주건축물,660.0,171.69,26.01,171.69,171.69,26.01,11.0,벽돌구조,"조적조, 경량철골조",01000,단독주택,"단독주택, 창고",90.0,기타지붕,판넬,0,1,4.2,1,0,0,0,1,72.60,99.09,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,20020619.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN,202102
3,28720-925,1,일반,2,일반건축물,인천광역시 옹진군 북도면 신도리 500번지,인천광역시 옹진군 북도면 신도로896번길 28,NaN,28720,31022.0,0.0,500,0,NaN,NaN,NaN,0,2.872043e+11,31001.0,0.0,28.0,0.0,NaN,0.0,주건축물,386.0,51.30,13.29,51.30,51.30,13.29,12.0,블록구조,블록구조,01000,단독주택,단독주택,30.0,슬레이트,슬레이트,0,1,0.0,1,0,0,0,0,0.00,51.30,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,19771218.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN,202102
4,28710-14164,1,일반,2,일반건축물,인천광역시 강화군 송해면 하도리 277-6번지,인천광역시 강화군 송해면 강화대로 819,NaN,28710,39026.0,0.0,277,6,NaN,NaN,NaN,0,2.871022e+11,39003.0,0.0,819.0,0.0,NaN,0.0,주건축물,226.0,70.03,30.99,70.03,70.03,30.99,32.0,경량철골구조,경량철골조,01000,단독주택,단독주택,90.0,기타지붕,판넬,0,1,4.0,1,0,0,0,0,0.00,70.03,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,20020321.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110414,NaN,NaN,202102


In [9]:
for i in range(len(dfs_list)):
    print(dfs_list[i].shape)

(7389045, 78)
(7563464, 78)
(7645979, 78)
(7725700, 76)
(7791811, 76)
(7837517, 78)
(7849930, 78)
(7483690, 78)


In [10]:
for i in range(len(dfs_list)):
    if dfs_list[i].shape[1] == 78:
        dfs_list[i] = dfs_list[i].drop(columns=['내진_능력', '내진_설계_적용_여부'])
    
    print(dfs_list[i].shape)

(7389045, 76)
(7563464, 76)
(7645979, 76)
(7725700, 76)
(7791811, 76)
(7837517, 76)
(7849930, 76)
(7483690, 76)


In [11]:
for i in range(len(dfs_list)):
    df = dfs_list[i]
    df['대지_위치'].fillna('없음', inplace=True)
    df = df[df['대지_위치'].str.contains('서울특별시')]
    dfs_list[i] = df

In [12]:
del df
gc.collect()

98

In [13]:
prac_df = dfs_list[0]

In [14]:
prac_df['관리_건축물대장_PK'] = prac_df['관리_건축물대장_PK'].apply(lambda x: x[:-2])

In [15]:
dfs_list[0] = prac_df

In [16]:
concat_df = pd.concat(dfs_list).reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

(4971419, 76)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,주_부속_구분_코드,주_부속_구분_코드_명,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,지붕_코드,지붕_코드_명,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월
0,11305-100193109,2,집합,3,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,11305,10300.0,0.0,484.0,80.0,NaN,NaN,NaN,1,1.130541e+11,10301.0,0.0,36.0,0.0,삼일교회,0.0,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,21.0,철근콘크리트구조,철근콘크리트구조(조적조(4층)),04000,제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",10.0,(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,3080079.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,201412
1,11440-21639,2,집합,3,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,11440,12000.0,0.0,484.0,16.0,NaN,NaN,NaN,0,1.144041e+11,12001.0,0.0,61.0,10.0,NaN,0.0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,21.0,철근콘크리트구조,철근콘크리트구조,02000,공동주택,다세대주택,10.0,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,3130085.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,201412
2,11200-23602,1,일반,2,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,11200,11800.0,0.0,73.0,566.0,NaN,NaN,NaN,0,1.120041e+11,11801.0,0.0,17.0,0.0,NaN,0.0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,21.0,철근콘크리트구조,"철근콘크리트,철골조",06000,종교시설,교회,10.0,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131,201412
3,11380-26906,1,일반,2,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,11380,10700.0,0.0,594.0,87.0,NaN,NaN,NaN,0,1.138030e+11,10702.0,0.0,163.0,13.0,NaN,0.0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,11.0,벽돌구조,연와조,01000,단독주택,"단독주택,다가구용 (6가구)",10.0,(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,201412
4,11110-100178925,1,일반,2,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,11110,18300.0,0.0,365.0,1.0,NaN,NaN,NaN,0,1.111041e+11,18301.0,0.0,45.0,0.0,NaN,0.0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,21.0,철근콘크리트구조,철근콘크리트구조,03000,제1종근린생활시설,제1종근린생활시설,10.0,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,3000082.0,건축과,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101,201412


In [17]:
def correct_addr(x):
    # x is a string
    if (pd.isna(x) == True)|(x == '없음'):
        return x
    else:
        x = x.replace('번지', '')
        if x[0] == ' ':
            for i in range(len(x)):
                if x[i] != ' ':
                    x = x[i:]
                    break
        
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [18]:
concat_df['지번주소'] = concat_df['대지_위치'].apply(correct_addr)

In [19]:
concat_df.drop(columns=[col for col in concat_df.columns if col.endswith('_코드')], inplace=True)
concat_df.shape

(4971419, 63)

In [20]:
concat_df.columns = [col.replace('_코드_명', '') for col in concat_df.columns]

In [21]:
concat_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,대지_위치,도로명_대지_위치,건물_명,번,지,특수지_명,블록,로트,외필지_수,새주소_본_번,새주소_부_번,동_명,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,세대_수(세대),가구_수(가구),높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관,허가번호_구분,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소
0,11305-100193109,집합,표제부,서울특별시 강북구 수유동 484-80번지,서울특별시강북구 삼양로77가길 36,삼일교회,484.0,80.0,NaN,NaN,NaN,1,36.0,0.0,삼일교회,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,철근콘크리트구조,철근콘크리트구조(조적조(4층)),제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",(철근)콘크리트,(철근)콘크리트(평스라브),3,0,12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20070821,20070829,20101216.0,2007,건축과,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101228,201412,서울특별시 강북구 수유동 484-80
1,11440-21639,집합,표제부,서울특별시 마포구 서교동 484-16번지,서울특별시마포구 양화로7길 61-10,NaN,484.0,16.0,NaN,NaN,NaN,0,61.0,10.0,NaN,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,4,0,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20030628,20030820,20040323.0,2003,건축과,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16
2,11200-23602,일반,일반건축물,서울특별시 성동구 송정동 73-566번지,서울특별시성동구 송정14길 17,대한예수교장로회비전교회,73.0,566.0,NaN,NaN,NaN,0,17.0,0.0,NaN,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,철근콘크리트구조,"철근콘크리트,철골조",종교시설,교회,(철근)콘크리트,"슬라브,경량철골조",0,0,13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,NaN,NaN,19781205.0,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110131,201412,서울특별시 성동구 송정동 73-566
3,11380-26906,일반,일반건축물,서울특별시 은평구 응암동 594-87번지,서울특별시은평구 응암로 163-13,NaN,594.0,87.0,NaN,NaN,NaN,0,163.0,13.0,NaN,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,벽돌구조,연와조,단독주택,"단독주택,다가구용 (6가구)",(철근)콘크리트,슬라브,0,6,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19900403,NaN,19911128.0,NaN,NaN,NaN,0,NaN,0.0,0,NaN,0,NaN,0,20110105,201412,서울특별시 은평구 응암동 594-87
4,11110-100178925,일반,일반건축물,서울특별시 종로구 평창동 365-1번지,서울특별시종로구 평창11길 45,NaN,365.0,1.0,NaN,NaN,NaN,0,45.0,0.0,NaN,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,철근콘크리트구조,철근콘크리트구조,제1종근린생활시설,제1종근린생활시설,(철근)콘크리트,(철근)콘크리트,0,0,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20070605,20070616,20080319.0,2007,건축과,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20110101,201412,서울특별시 종로구 평창동 365-1


In [22]:
concat_df['구조'].unique()

array(['철근콘크리트구조', '벽돌구조', '일반철골구조', '기타조적구조', '일반목구조', '블록구조',
       '철골철근콘크리트구조', '경량철골구조', '철골콘크리트구조', '석구조', nan, '기타콘크리트구조',
       '강파이프구조', '프리케스트콘크리트구조', '기타강구조', '기타구조', '기타철골철근콘크리트구조', '조적구조',
       '통나무구조', '목구조', '철골철근콘크리트합성구조', '조립식판넬조', '시멘트블럭조', '라멘조',
       '스틸하우스조', '컨테이너조'], dtype=object)

In [23]:
concat_df.drop(columns=['대지_위치', '도로명_대지_위치', '건물_명', '번', '지', '특수지_명', '블록', '로트', '새주소_본_번',
                        '새주소_부_번', '동_명', '허가번호_기관', '허가번호_구분'
                       ], inplace=True)

In [24]:
concat_df['총세대수'] = concat_df['세대_수(세대)'] + concat_df['가구_수(가구)'] + concat_df['호_수(호)']
concat_df.drop(columns=['세대_수(세대)', '가구_수(가구)', '호_수(호)'], inplace=True)

In [25]:
bdinfo_df = concat_df
del concat_df

In [26]:
def apply_int(x):
    # x is a number or a string
    
    if pd.isna(x) == True:
        return x
    else:
        if type(x) == str:
            try:
                return int(x.replace(' ', '0'))
            except:
                raise ValueError(x)
        else:
            return int(x)

In [27]:
bdinfo_df['사용승인_일_int'] = bdinfo_df['사용승인_일'].apply(apply_int)

In [28]:
bdinfo_df.drop(columns=['허가_일', '착공_일', '허가번호_년'], inplace=True)

In [29]:
bdinfo_df['사용승인_일'] = bdinfo_df['사용승인_일_int'].astype('Int64')

In [30]:
bdinfo_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,사용승인_일_int
0,11305-100193109,집합,표제부,1,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,철근콘크리트구조,철근콘크리트구조(조적조(4층)),제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",(철근)콘크리트,(철근)콘크리트(평스라브),12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20101216,NaN,0.0,0,NaN,0,NaN,0,20101228,201412,서울특별시 강북구 수유동 484-80,3,20101216.0
1,11440-21639,집합,표제부,0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,20040323.0
2,11200-23602,일반,일반건축물,0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,철근콘크리트구조,"철근콘크리트,철골조",종교시설,교회,(철근)콘크리트,"슬라브,경량철골조",13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,19781205,NaN,0.0,0,NaN,0,NaN,0,20110131,201412,서울특별시 성동구 송정동 73-566,0,19781205.0
3,11380-26906,일반,일반건축물,0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,벽돌구조,연와조,단독주택,"단독주택,다가구용 (6가구)",(철근)콘크리트,슬라브,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19911128,NaN,0.0,0,NaN,0,NaN,0,20110105,201412,서울특별시 은평구 응암동 594-87,6,19911128.0
4,11110-100178925,일반,일반건축물,0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,철근콘크리트구조,철근콘크리트구조,제1종근린생활시설,제1종근린생활시설,(철근)콘크리트,(철근)콘크리트,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20080319,NaN,0.0,0,NaN,0,NaN,0,20110101,201412,서울특별시 종로구 평창동 365-1,0,20080319.0


In [31]:
bdinfo_df.drop(columns=['사용승인_일_int'], inplace=True)

In [32]:
bdinfo_df['addr_split_len'] = bdinfo_df['지번주소'].apply(lambda x: len(x.split(' ')))

In [33]:
bdinfo_df['addr_split_len'].value_counts()

4    4929105
5      38026
6       2836
7       1138
8        172
3        123
9         19
Name: addr_split_len, dtype: int64

In [34]:
len3_df = bdinfo_df[bdinfo_df['addr_split_len'] == 3]
bdinfo_df.drop(len3_df.index, inplace=True)
len9_df = bdinfo_df[bdinfo_df['addr_split_len'] == 9]
bdinfo_df.drop(len9_df.index, inplace=True)
len8_df = bdinfo_df[bdinfo_df['addr_split_len'] == 8]
bdinfo_df.drop(len8_df.index, inplace=True)
len7_df = bdinfo_df[bdinfo_df['addr_split_len'] == 7]
bdinfo_df.drop(len7_df.index, inplace=True)
len6_df = bdinfo_df[bdinfo_df['addr_split_len'] == 6]
bdinfo_df.drop(len6_df.index, inplace=True)

In [35]:
len5_df = bdinfo_df[bdinfo_df['addr_split_len'] == 5]

In [36]:
len5_df['fourth_addr'] = len5_df['지번주소'].apply(lambda x: x.split(' ')[3])

In [37]:
bdinfo_df['대장구분명'] = np.nan

In [38]:
def landtype_name(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if x == '산':
            return '산'
        elif ('블록' in x)&('지구' in x):
            return '지구블록'
        elif '블록' in x:
            return 'BL'

In [39]:
len5_df['대장구분명'] = len5_df['fourth_addr'].apply(landtype_name)

In [40]:
bdinfo_df.loc[len5_df.index, '대장구분명'] = len5_df['대장구분명']

In [41]:
def weird_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        try:
            int(x.split(' ')[-1].replace('-', ''))
            return 0
        except:
            return 1

In [42]:
len4_df = bdinfo_df[bdinfo_df['addr_split_len'] == 4]
len4_df['weird_addr'] = len4_df['지번주소'].apply(weird_addr)

In [43]:
weird_addr_df = len4_df[len4_df['weird_addr'] == 1]

In [44]:
bdinfo_df.drop(weird_addr_df.index, inplace=True)

In [45]:
bdinfo_df = bdinfo_df[bdinfo_df['대장구분명'] != '지구블록']

In [46]:
bdinfo_df = bdinfo_df[bdinfo_df['대장구분명'] != 'BL']

In [47]:
bdinfo_df['대장구분명'].fillna('일반', inplace=True)
bdinfo_df['대장구분명'].value_counts()

일반    4928196
산       35185
Name: 대장구분명, dtype: int64

In [48]:
bdinfo_df.drop(columns=['addr_split_len'], inplace=True)

In [49]:
bdinfo_san_df = bdinfo_df[bdinfo_df['대장구분명'] == '산']

In [50]:
def san_new_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split(' ')
        return splitted[0] + ' ' + splitted[1] + ' ' + splitted[2] + ' ' + splitted[3] + splitted[4]

In [51]:
bdinfo_san_df['지번주소'] = bdinfo_san_df['지번주소'].apply(san_new_addr)

In [52]:
bdinfo_df.loc[bdinfo_san_df.index, '지번주소'] = bdinfo_san_df['지번주소']

In [53]:
def addr_split_len(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        return len(x.split(' '))

In [54]:
bdinfo_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명
0,11305-100193109,집합,표제부,1,주건축물,393.00,233.81,59.49,761.29,761.29,193.71,철근콘크리트구조,철근콘크리트구조(조적조(4층)),제2종근린생활시설,"근린생활시설, 공동주택(다세대주택)",(철근)콘크리트,(철근)콘크리트(평스라브),12.2,4,0,0,0,0,0.0,761.29,0,0.0,0,0.0,0,0.00,7,80.50,20101216,NaN,0.0,0,NaN,0,NaN,0,20101228,201412,서울특별시 강북구 수유동 484-80,3,일반
1,11440-21639,집합,표제부,0,주건축물,177.62,105.30,59.28,393.13,385.21,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.0,4,1,0,0,0,0.0,393.13,0,0.0,0,0.0,0,0.00,4,46.00,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반
2,11200-23602,일반,일반건축물,0,주건축물,806.00,376.05,46.66,1185.07,797.98,99.00,철근콘크리트구조,"철근콘크리트,철골조",종교시설,교회,(철근)콘크리트,"슬라브,경량철골조",13.2,2,1,0,0,0,0.0,1185.07,0,0.0,0,0.0,0,0.00,0,0.00,19781205,NaN,0.0,0,NaN,0,NaN,0,20110131,201412,서울특별시 성동구 송정동 73-566,0,일반
3,11380-26906,일반,일반건축물,0,주건축물,0.00,0.00,0.00,246.15,0.00,0.00,벽돌구조,연와조,단독주택,"단독주택,다가구용 (6가구)",(철근)콘크리트,슬라브,0.0,2,1,0,0,0,0.0,246.15,0,0.0,0,0.0,0,0.00,0,0.00,19911128,NaN,0.0,0,NaN,0,NaN,0,20110105,201412,서울특별시 은평구 응암동 594-87,6,일반
4,11110-100178925,일반,일반건축물,0,주건축물,356.00,104.22,29.28,404.80,194.31,54.58,철근콘크리트구조,철근콘크리트구조,제1종근린생활시설,제1종근린생활시설,(철근)콘크리트,(철근)콘크리트,7.9,2,1,0,0,0,0.0,404.80,0,0.0,0,0.0,1,33.29,2,40.19,20080319,NaN,0.0,0,NaN,0,NaN,0,20110101,201412,서울특별시 종로구 평창동 365-1,0,일반


In [55]:
main_data_df = pd.read_csv('./prepped_data/메인_데이터_20060101_20210416_ver_3.csv')
print(main_data_df.shape)
main_data_df.head()

(1903435, 18)


,지번주소,도로명주소,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,건물연식,부동산유형,전용면적_classes,가격_면적,가격_면적_classes,X좌표,Y좌표,구,동
0,서울특별시 강남구 개포동 1264-3,서울특별시 강남구 개포로31길 23-7,(1264-3),53.28,11500,-1,1992.0,14.0,2006-08-29,29.0,연립다세대,50~55,215.840841,210~220,960076.154238,1.942459e+06,강남구,개포동
1,서울특별시 강남구 개포동 171-13,서울특별시 강남구 선릉로14길 11-0,(171-13),68.08,56500,2,1988.0,18.0,2006-12-20,33.0,연립다세대,65~70,829.905993,820~830,961145.462320,1.942843e+06,강남구,개포동
2,서울특별시 강남구 개포동 171-13,서울특별시 강남구 선릉로14길 11-0,(171-13),44.80,23300,-1,1988.0,28.0,2016-01-16,33.0,연립다세대,40~45,520.089286,520~530,961145.462320,1.942843e+06,강남구,개포동
3,서울특별시 강남구 개포동 171-13,서울특별시 강남구 선릉로14길 11-0,(171-13),44.80,24500,-1,1988.0,28.0,2016-04-09,33.0,연립다세대,40~45,546.875000,540~550,961145.462320,1.942843e+06,강남구,개포동
4,서울특별시 강남구 개포동 171-13,서울특별시 강남구 선릉로14길 11-0,(171-13),68.08,60000,2,1988.0,31.0,2019-10-23,33.0,연립다세대,65~70,881.316099,880~890,961145.462320,1.942843e+06,강남구,개포동


In [56]:
bdinfo_df.shape

(4963381, 46)

In [57]:
bdinfo_df['사용승인_일'].isna().sum()

287602

In [58]:
bdinfo_df.dropna(subset=['사용승인_일'], inplace=True)

In [59]:
bdinfo_df['건축년도'] = bdinfo_df['사용승인_일'].apply(lambda x: int(str(x)[:4]))

In [60]:
selected_df = bdinfo_df[['지번주소', '건축년도', '외필지_수']].drop_duplicates()
print(selected_df.shape)
selected_df.head()

(603493, 3)


,지번주소,건축년도,외필지_수
0,서울특별시 강북구 수유동 484-80,2010,1
1,서울특별시 마포구 서교동 484-16,2004,0
2,서울특별시 성동구 송정동 73-566,1978,0
3,서울특별시 은평구 응암동 594-87,1991,0
4,서울특별시 종로구 평창동 365-1,2008,0


In [61]:
zero_df = selected_df[selected_df['외필지_수'] == 0]
print(zero_df.shape)
zero_df.head()

(550228, 3)


,지번주소,건축년도,외필지_수
1,서울특별시 마포구 서교동 484-16,2004,0
2,서울특별시 성동구 송정동 73-566,1978,0
3,서울특별시 은평구 응암동 594-87,1991,0
4,서울특별시 종로구 평창동 365-1,2008,0
5,서울특별시 강북구 수유동 30-8,2003,0


In [63]:
zero_df['지번주소_건축년도'] = zero_df['지번주소'] + '_' + zero_df['건축년도'].apply(str)

In [64]:
zero_df['지번주소_건축년도'].nunique()

550228

In [65]:
zero_df.drop(columns=['지번주소_건축년도'], inplace=True)
print(zero_df.shape)
zero_df.head()

(550228, 3)


,지번주소,건축년도,외필지_수
1,서울특별시 마포구 서교동 484-16,2004,0
2,서울특별시 성동구 송정동 73-566,1978,0
3,서울특별시 은평구 응암동 594-87,1991,0
4,서울특별시 종로구 평창동 365-1,2008,0
5,서울특별시 강북구 수유동 30-8,2003,0


In [67]:
zero_df['건축년도'].value_counts()

1991    28846
1993    28630
1990    24462
1992    22542
1994    20351
1989    20072
1988    17950
1985    15885
2002    15246
1996    14739
1995    14034
1997    13365
1986    12363
1987    11833
2003    11697
1983    10943
1974    10749
1977     9937
1975     9562
1970     9524
1976     9398
1984     9235
1971     9180
1978     9106
1973     9025
2001     7842
2004     7573
1969     6795
1968     6127
1967     5816
1980     5805
1982     5593
1972     5452
2016     5392
2017     5370
1998     5207
2008     5015
1981     4839
2018     4791
2012     4773
2000     4485
2011     4483
2015     4465
1966     4312
2006     4299
2014     4274
2007     4270
1979     4170
2010     4058
2013     4029
1961     3938
2019     3912
2005     3673
1960     3442
1965     3436
2009     3125
2020     2959
1959     2819
1999     2713
1964     2084
1958     1943
1963     1758
1957     1424
1956     1384
1955     1375
1962     1149
1954     1065
1936      771
1937      759
1949      741
1948      550
1940  

In [66]:
%%time
zero_df.to_csv('./prepped_data/표제부_외필지_수_0.csv', index=False)

Wall time: 664 ms
